In [2]:
import cv2

# Create a VideoCapture object for the input video
capture = cv2.VideoCapture("Video2.mp4")

# Check if the video was opened successfully
if not capture.isOpened():
    print("Error opening video file")

# Create a background subtractor object
fgbg = cv2.createBackgroundSubtractorMOG2(history=10)

# Set the threshold for the foreground mask
threshold = 20

# Define the frame width and height
frame_width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create a VideoWriter object for the output video
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
writer = cv2.VideoWriter("MOG_Tracker.avi", fourcc, 30.0, (frame_width, frame_height))

# Read the frames from the video
while capture.isOpened():
    # Read the next frame
    success, frame = capture.read()

    # Check if the frame was read successfully
    if not success:
        break

    # Apply the background subtractor to the frame
    fgmask = fgbg.apply(frame)

    # Threshold the foreground mask to create a binary mask
    _, mask = cv2.threshold(fgmask, threshold, 255, cv2.THRESH_BINARY)

    # Find the contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Iterate over the contours
    for cnt in contours:
        # Check if the contour area is above a certain threshold
        area = cv2.contourArea(cnt)
        if area > 13:
            # Draw the contour in green on the frame
            cv2.drawContours(frame, [cnt], -1, (0, 255, 0), 2)

    # Write the frame to the output video
    writer.write(frame)

# Release the VideoCapture and VideoWriter objects
capture.release()
writer.release()